# Detecting Fraud on Ethereum

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import data from data/transactions.csv
transactions_df = pd.read_csv('data/transactions.csv')

# show summary statistics
transactions_df.describe()

,nonce,block_number,transaction_index,gas,gas_price,block_timestamp,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,max_fee_per_blob_gas,blob_versioned_hashes
count,139255.000000,139255.000000,139255.000000,1.392550e+05,1.392550e+05,1.392550e+05,0.0,0.0,139255.0,0.0,0.0
mean,5754.232875,177011.407253,17.611784,7.571893e+04,8.028699e+10,1.441205e+09,NaN,NaN,0.0,NaN,NaN
std,7514.660700,36888.037772,31.373293,2.242644e+05,4.544315e+11,6.933802e+05,NaN,NaN,0.0,NaN,NaN
min,0.000000,100004.000000,0.000000,2.100000e+04,4.500000e+10,1.439799e+09,NaN,NaN,0.0,NaN,NaN
25%,29.000000,148528.000000,0.000000,2.100000e+04,5.000000e+10,1.440635e+09,NaN,NaN,0.0,NaN,NaN
50%,1237.000000,178829.000000,2.000000,2.500000e+04,5.000000e+10,1.441263e+09,NaN,NaN,0.0,NaN,NaN
75%,10929.000000,207847.000000,18.000000,9.000000e+04,5.466458e+10,1.441799e+09,NaN,NaN,0.0,NaN,NaN
max,27670.000000,238199.000000,148.000000,3.141592e+06,1.000000e+13,1.442326e+09,NaN,NaN,0.0,NaN,NaN


In [2]:
# create a copy of the df to explore and manipulate #
df = transactions_df.copy()

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139255 entries, 0 to 139254
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   hash                      139255 non-null  object 
 1   nonce                     139255 non-null  int64  
 2   block_hash                139255 non-null  object 
 3   block_number              139255 non-null  int64  
 4   transaction_index         139255 non-null  int64  
 5   from_address              139255 non-null  object 
 6   to_address                138322 non-null  object 
 7   value                     139255 non-null  object 
 8   gas                       139255 non-null  int64  
 9   gas_price                 139255 non-null  int64  
 10  input                     139255 non-null  object 
 11  block_timestamp           139255 non-null  int64  
 12  max_fee_per_gas           0 non-null       float64
 13  max_priority_fee_per_gas  0 non-null       f

,hash,nonce,block_hash,block_number,transaction_index,from_address,to_address,value,gas,gas_price,input,block_timestamp,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,max_fee_per_blob_gas,blob_versioned_hashes
0,0xd133956b0ddcd58f30fcc60ed18d6f175b03b342b203...,164,0x9493fb89a5bd30ca0c49f9d821715b7eec06d6007aa1...,100405,0,0x007f7f58d3eb5b7510a301ecc749fc1fcddbe14d,0xb1abce2918e21ddb93aa452731a12672a3d9f75a,5000000000000000000,90000,60347544134,0x,1439806029,NaN,NaN,0,NaN,NaN
1,0x174b4f8118e1429581e9d8154c204f3c6d355b059971...,11,0x3c6e938bc68872a0f3b12a6226ba3da21fecd81a17c6...,100408,0,0xcd1e66ed539dd92fc40bbaa1fa16de8c02c14d45,0xe0e82ed3e93079ce76aac1da7b4bfa990d885a0d,174420874796219315407,21000,55854227910,0x,1439806081,NaN,NaN,0,NaN,NaN
2,0xba2f997aa470c83fbccf71f8f8d59dc972a4cd39e70c...,15,0x44e25aa76eb0a02fb369c20a08553b34121c2e842398...,100409,0,0x0a319a7e0c887f104359e1eee6717bd8f232d3da,0x32be343b94f860124dc4fee278fdcbd38c102d88,1780494800000000000,21000,59043039037,0x,1439806096,NaN,NaN,0,NaN,NaN
3,0x82776456a7f8d881eb28a29d4e4f898028742906e30f...,6,0x44e25aa76eb0a02fb369c20a08553b34121c2e842398...,100409,1,0x3b399ff2b96f27056ab8a7f6e028b0dbaf386279,0x32be343b94f860124dc4fee278fdcbd38c102d88,29277820729999998976,21000,59043039037,0x,1439806096,NaN,NaN,0,NaN,NaN
4,0xce9894a6cd506133f98c23781a2228c27ae1edc41648...,29,0x44e25aa76eb0a02fb369c20a08553b34121c2e842398...,100409,2,0xcf00a85f3826941e7a25bfcf9aac575d40410852,0xd9666150a9da92d9108198a4072970805a8b3428,5000000000000000000,90000,54588778004,0x,1439806096,NaN,NaN,0,NaN,NaN


In [3]:
# max_fee_per_gas & max_priority_fee_per_gas have no non null entries, this feature was added in the London Hardfork (Aug 2021) upgrade
# dropping these columns is ok
london_hardfork_columns = df[['max_fee_per_gas', 'max_priority_fee_per_gas']].isna().value_counts()
print(f"Number of entries with null max_fee_per_gas or max_priority_fee_per_gas: {london_hardfork_columns.sum()}")
df = df.drop(columns=['max_fee_per_gas', 'max_priority_fee_per_gas'])

# max_fee_per_blob_gas & blob_versioned_hashes have no non null entries, this feature will be added in the Proto-Danksharding upgrade
# dropping these columns is ok
danksharding_columns = df[['max_fee_per_blob_gas', 'blob_versioned_hashes']].isna().value_counts()
print(f"Number of entries with null max_fee_per_blob_gas or blob_versioned_hashes: {danksharding_columns.sum()}")
df = df.drop(columns=['max_fee_per_blob_gas', 'blob_versioned_hashes'])

Number of entries with null max_fee_per_gas or max_priority_fee_per_gas: 139255
Number of entries with null max_fee_per_blob_gas or blob_versioned_hashes: 139255


In [4]:
## UPDATED DATASET HAS NO MISSING VALUES FOR TO_ADDRESS AND FROM ADDRESS

# exploration of columns with null/missing values:

# to_address is missing 5/500 values, receipt_contract_address is missing 495/500
# exploration to see if this indicates directionality of the transaction

# null_to_address = df[df['to_address'].isnull()].index.tolist()
# null_receipt_address = df[df['from_address'].isnull()].index.tolist()
# print(f"Entries missing to_address: {null_to_address}")

# print("Entries missing both to_address and from_address:" + str([x for x in null_receipt_address if x in null_to_address]) + '\n')
# for idx in null_to_address:
#     receipt_address = df['from_address'].iloc[idx]
#     print(f"Receipt contract address for entry missing to_address: {receipt_address}")

# print("Number of missing receipt addresses: " + str(len(null_receipt_address)))
# print("Entries missing a to_address have a from_address, entries missing a receipt_contract_address have a to_address, these features are mutually exclusive.")

In [5]:
# Consider usable formats for features #

# Hash is a string used to identify transactions - can use index instead (?)

# what is nonce? what can this tell us about each transaction?

# BD: it's a unique number that tracks the number of transactions sent from a given address. 
# Each Ethereum account has a nonce value that increases with each transaction the account sends

# the max nonce value below shows that at least one account has sent 27671 transactions in the dataset

# HR: A question we may wish to address is whether accounts with smaller nonce are more/less likely to experience fraud? Intuition would have me think
# that accounts that make lots of transaction are "safe" - or on the flip side maybe accounts with high nonce over a short period are more likely to be fraudulent

print(f"Max nonce: {df['nonce'].max()}")
print(f"Min nonce: {df['nonce'].min()}")
print(f"Mean nonce: {df['nonce'].mean()}")
print(f"Number of unique nonce values: {df['nonce'].nunique()}")

time_example = df['block_timestamp'][0]
print(f"Timestamp example entry 0: {time_example}")
timestamp_format = pd.to_datetime(time_example, unit='s')
print(f"Using timestamp format from Unix epoch: {timestamp_format}")

# Create new column to see block_timestamp in datetime format:
df['block_timestamp_dt'] = pd.to_datetime(df['block_timestamp'], unit='s')
# Create new columns to record the year, month, day, and time - good for human readability may not be useful for analysis
df['transaction_year'] = df['block_timestamp_dt'].dt.year
df['transaction_month'] = df['block_timestamp_dt'].dt.month
df['transaction_day'] = df['block_timestamp_dt'].dt.day
df['transaction_time'] = df['block_timestamp_dt'].dt.time

print(df.info())

Max nonce: 27670
Min nonce: 0
Mean nonce: 5754.232874941654
Number of unique nonce values: 27671
Timestamp example entry 0: 1439806029
Using timestamp format from Unix epoch: 2015-08-17 10:07:09
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139255 entries, 0 to 139254
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   hash                139255 non-null  object        
 1   nonce               139255 non-null  int64         
 2   block_hash          139255 non-null  object        
 3   block_number        139255 non-null  int64         
 4   transaction_index   139255 non-null  int64         
 5   from_address        139255 non-null  object        
 6   to_address          138322 non-null  object        
 7   value               139255 non-null  object        
 8   gas                 139255 non-null  int64         
 9   gas_price           139255 non-null  int64         
 10  input

In [6]:
# explore wallets that receive lots of transactions:

print(f"Number of unique to_addresses: {df['to_address'].nunique()}")
print(f"Frequency of to_addresses: \n{df['to_address'].value_counts()}")

# The majority of transactions are sent to address '0x9af09991ad63814e53ffc1bccf213ee74027608b'

Number of unique to_addresses: 6754
Frequency of to_addresses: 
0x32be343b94f860124dc4fee278fdcbd38c102d88    38797
0x7c5080988c6d91d090c23d54740f856c69450b29     7180
0xc47aaa860008be6f65b58c6c6e02a84e666efe31     3292
0x58850ab9442c06379dffbf52e3eaa035f5d5e092     2824
0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98     2597
                                              ...  
0x89e696a97d69f05d78e3ebf78c92092c1cc80e3f        1
0x1706bf774949d89737d0723bee3ef3511ab30dad        1
0x601b2281bde506e33c74a57917e6e2d48476697e        1
0x126cc7c723d495f2f9862746573477901fdff58a        1
0xabd8567b813b501034a36b8053eb4bf0b60e4fce        1
Name: to_address, Length: 6754, dtype: int64
